In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.bd_rede_hyundai 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.bd_rede_hyundai;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.campaigns 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.campaigns;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.campanha_vouchers
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.campanha_vouchers; 

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.campanhas_tecnicas 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.campanhas_tecnicas;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contactos_importador_pbs 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contactos_importador_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contactos_pbs 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contactos_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contas_importador_pbs 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contas_importador_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contas_pbs 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contas_pbs;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.contratos_financiamento 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.contratos_financiamento;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.deals
SELECT * EXCEPT(created_time),
  		 to_timestamp(created_time, 'dd-MM-yyyy HH:mm') as created_time
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.deals;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.historico_de_servicos 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.historico_de_servicos;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.leads_pbs 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.leads_pbs;

In [0]:
%sql
SELECT pvp as original_value,

       regexp_replace(regexp_replace(regexp_replace(pvp, '[€%\\p{Z}]', ''),   --\\p{Z} matches every space/separator in Unicode
                                                                           '[,.](?=.{3,})', ''),--Replaces any comma/period (with at least 3 characters afterwards) with empty string
                                                                           ',([0-9]{2})$', '.\1'--Replaces the matched comma (before the 2 digits at the end) with a period.
                     ) AS cleaned_value,

      cast(clean_for_decimal_type(pvp) AS DECIMAL(29,2)) as pvp,

      COUNT(*) as occurrence_count

FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.propostas_realizadas

WHERE pvp IS NOT NULL 
  AND regexp_replace(pvp, '[€%\\p{Z}]', '') not IN ('', 'null', 'NaN') 
  AND pvp NOT RLIKE '^[€%,.\\p{Z}]+$'

GROUP BY pvp

ORDER BY cleaned_value DESC;

In [0]:
%sql
CREATE OR REPLACE FUNCTION clean_for_decimal_type(column_value string) RETURNS DECIMAL(29,2)

RETURN CASE 
    WHEN column_value IS NULL THEN NULL
    WHEN regexp_replace(column_value, '[€%\\p{Z}]', '') IN ('', 'null', 'NaN') --First, remove every space/separator in Unicode, in addition to % and €
      OR column_value RLIKE '^[€%,.\\p{Z}]+$' THEN NULL --It consists of '[€%,.\\p{Z}]' and nothing else
    
    ELSE CAST(regexp_replace(regexp_replace(regexp_replace(column_value, '[€%\\p{Z}]', ''),
                                                                         '[,.](?=.{3,})', ''),--Replaces any comma/period (with at least 3 characters afterwards) with empty string
                                                                         ',([0-9]{2})$', '.\1'--Replaces the matched comma (before the 2 digits at the end) with a period.
             ) AS DECIMAL(29,2))
END;


In [0]:
%sql
CREATE OR REPLACE FUNCTION clean_for_timestamp_type(column_value string) RETURNS TIMESTAMP

RETURN CASE
  WHEN column_value IS NULL THEN NULL
  WHEN regexp_replace(column_value, '[€%\\p{Z}]', '') IN ('', 'null', 'NaN') --First, remove every space/separator in Unicode, in addition to % and €
    OR column_value RLIKE '^[€%,.\\p{Z}]+$' THEN NULL --It consists of '[€%,.\\p{Z}]' and nothing else
  
	WHEN column_value RLIKE '^\\d{4}-\\d{2}-\\d{2} \\d{2}:\\d{2}$' THEN try_to_timestamp(column_value, 'yyyy-MM-dd HH:mm') --try_to_timestamp() safely returns NULL for invalid dates
	WHEN column_value RLIKE '^\\d{2}-\\d{2}-\\d{4} \\d{2}:\\d{2}$' THEN try_to_timestamp(column_value, 'dd-MM-yyyy HH:mm')
	
  ELSE NULL --No rows match this condition. PS: Since ELSE is mandatory, we always confirm beforehand
END;

In [0]:
%sql
CREATE OR REPLACE FUNCTION clean_for_date_type(column_value string) RETURNS DATE

RETURN CASE
  WHEN column_value IS NULL THEN NULL
  WHEN regexp_replace(column_value, '[€%\\p{Z}]', '') IN ('', 'null', 'NaN') --First, remove every space/separator in Unicode, in addition to % and €
    OR column_value RLIKE '^[€%,.\\p{Z}]+$' THEN NULL --It consists of '[€%,.\\p{Z}]' and nothing else

  WHEN column_value RLIKE '^\\d{4}-\\d{2}-\\d{2}$' THEN to_date(column_value, 'yyyy-MM-dd')
	WHEN column_value RLIKE '^\\d{2}-\\d{2}-\\d{4}$' THEN to_date(column_value, 'dd-MM-yyyy')
	
  ELSE NULL
END;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.propostas_realizadas

	    -- ID columns
SELECT  cast(_fivetran_id as bigint) as _fivetran_id,
	    cast(id as bigint) as id,
	    
		-- Basic info columns (containing Dates with validation)
	    cast(clean_for_timestamp_type(created_time) AS TIMESTAMP) AS created_time,
		cast(subject as string) as subject,
		cast(proposta_realizada_owner as bigint) as proposta_realizada_owner,
		cast(proposta_realizada_owner_name as string) as proposta_realizada_owner_name,
		cast(proposta_realizada_stage as string) as proposta_realizada_stage,
		cast(clean_for_timestamp_type(modified_time) AS TIMESTAMP) AS modified_time,
		
		-- Contact and account info
		cast(conta_name as string) as conta_name,
		cast(id_contacto as bigint) as id_contacto,
		cast(contacto_name as bigint) as contacto_name,
		cast(deal_name as bigint) as deal_name,
		
		-- Financial columns (removing € symbol, handling spaces in negative values, handling European decimal format)
		cast(clean_for_decimal_type(sub_total) AS DECIMAL(29,2)) AS sub_total,
		cast(clean_for_decimal_type(grand_total) AS DECIMAL(29,2)) AS grand_total,
		cast(created_by as bigint) as created_by,
		cast(modified_by as bigint) as modified_by,
		cast(concessao as string) as concessao,
		cast(clean_for_decimal_type(preco_base) AS DECIMAL(29,2)) AS preco_base,
		cast(clean_for_decimal_type(isv) AS DECIMAL(29,2)) AS isv,
		cast(clean_for_decimal_type(pintura) AS DECIMAL(29,2)) AS pintura,
		cast(clean_for_decimal_type(base_tributavel) AS DECIMAL(29,2)) AS base_tributavel,
		cast(clean_for_decimal_type(legalizacao) AS DECIMAL(29,2)) AS legalizacao,
		cast(clean_for_decimal_type(desconto_total) AS DECIMAL(29,2)) AS desconto_total,
		cast(clean_for_decimal_type(iuc) AS DECIMAL(29,2)) AS iuc,
		cast(clean_for_decimal_type(preco_total) AS DECIMAL(29,2)) AS preco_total,
		cast(iva as int) as iva,
		
		-- Vehicle info
		cast(modelo as string) as modelo,
		cast(descricao_apoio as string) as descricao_apoio,
		
		cast(clean_for_decimal_type(apoio_concessionario) as DECIMAL(29,2)) as apoio_concessionario,
		
		cast(clean_for_decimal_type(apoio_hyundai_portugal) AS DECIMAL(29,2)) AS apoio_hyundai_portugal,
		cast(clean_for_decimal_type(apoio_total) AS DECIMAL(29,2)) AS apoio_total,
		cast(clean_for_decimal_type(sub_total_com_extras) AS DECIMAL(29,2)) AS sub_total_com_extras,
		cast(clean_for_decimal_type(valor_extras) AS DECIMAL(29,2)) AS valor_extras,
		cast(clean_for_decimal_type(sub_total_com_desconto) AS DECIMAL(29,2)) AS sub_total_com_desconto,
		cast(clean_for_decimal_type(taxa) AS DECIMAL(29,2)) AS taxa,
		cast(clean_for_decimal_type(total_final) AS DECIMAL(29,2)) AS total_final,
		
		-- Status and proposal info
		cast(estado_do_pedido as string) as estado_do_pedido,
		cast(clean_for_date_type(data_de_criacao_da_proposta) AS DATE) AS data_de_criacao_da_proposta,
		
		cast(s_g_p_u_ as string) as s_g_p_u_,
		cast(codigo_da_proposta as string) as codigo_da_proposta,
		cast(descricao_pintura as string) as descricao_pintura,
		
		cast(clean_for_decimal_type(valor_do_apoio_pedido) AS DECIMAL(29,2)) as valor_do_apoio_pedido,	
		cast(clean_for_decimal_type(pvp) AS DECIMAL(29,2)) as pvp,

		cast(clean_for_date_type(data_de_entrega_da_proposta) AS DATE) AS data_de_entrega_da_proposta,
		cast(id_conta as bigint) as id_conta,
		cast(id_negocio as bigint) as id_negocio,
		cast(id_proposta_realizada as bigint) as id_proposta_realizada,
		
		--TBD in Python
		cast(data_de_validade_da_proposta as string) as data_de_validade_da_proposta,

		cast(clean_for_decimal_type(desconto_total__c__apoio_de_importador_) AS DECIMAL(29,2)) as desconto_total__c__apoio_de_importador_,

		-- Vehicle specification codes
		cast(codigo_modelo as string) as codigo_modelo,
		cast(sufixo_modelo as string) as sufixo_modelo,
		cast(codigo_cor_exterior as bigint) as codigo_cor_exterior,
		cast(codigo_cor_interior as bigint) as codigo_cor_interior,
		cast(concessionario_owner as string) as concessionario_owner,
		cast(clean_for_decimal_type(valor_aprovado) AS DECIMAL(29,2)) AS valor_aprovado,
		
		-- Dates
		cast(clean_for_date_type(data_prevista_matricula) AS DATE) AS data_prevista_matricula,
		cast(clean_for_date_type(data_prevista_de_entrega) AS DATE) AS data_prevista_de_entrega,
		cast(clean_for_date_type(data_da_conclusao) AS DATE) AS data_da_conclusao,
		
		-- Installation and keys
		cast(instalacao as string) as instalacao,
		cast(chave_instalacao_bd_rede as string) as chave_instalacao_bd_rede,
		cast(chave_concessao_bd_rede as string) as chave_concessao_bd_rede,
		cast(qtd_viaturas as int) as qtd_viaturas,
		cast(versao as string) as versao,
		cast(cod_proposta as string) as cod_proposta,
		cast(codigo_fabricante_cor_interior as string) as codigo_fabricante_cor_interior,
		cast(codigo_fabricante_cor_exterior as string) as codigo_fabricante_cor_exterior,
		cast(estado_ordering as string) as estado_ordering,
		
		-- Classification info
		cast(id_classe as string) as id_classe,
		cast(descricao_classe as string) as descricao_classe,
		cast(id_model_group as string) as id_model_group,
		cast(descricao_model_group as string) as descricao_model_group,
		cast(chave_agrupamento_cliente as string) as chave_agrupamento_cliente,
		cast(tipo_cliente as string) as tipo_cliente,
		cast(combustivel as string) as combustivel,
		cast(id_combustivel as int) as id_combustivel,
		cast(id_modelo as int) as id_modelo,
		cast(tylacode as string) as tylacode,
		
		-- Campaign and support values
		cast(clean_for_decimal_type(valor_campanha) AS DECIMAL(29,2)) AS valor_campanha,
		cast(clean_for_decimal_type(apoio_campanha_hyundai) AS DECIMAL(29,2)) AS apoio_campanha_hyundai,
		cast(clean_for_decimal_type(apoio_financiamento_cetelem) AS DECIMAL(29,2)) AS apoio_financiamento_cetelem,
		cast(clean_for_decimal_type(apoio_retoma_hyundai) AS DECIMAL(29,2)) AS apoio_retoma_hyundai,
		cast(clean_for_decimal_type(comparticipacao_campanha_da_concessao) AS DECIMAL(29,2)) AS comparticipacao_campanha_da_concessao,
		cast(clean_for_decimal_type(comparticipacao_retoma_concessao) AS DECIMAL(29,2)) AS comparticipacao_retoma_concessao,
		
		-- Additional info
		cast(forma_de_pagamento as string) as forma_de_pagamento,
		cast(categoria_proposta as string) as categoria_proposta,
		cast(gestor_area as string) as gestor_area,
		cast(clean_for_decimal_type(margem_frota_eni) AS DECIMAL(29,2)) AS margem_frota_eni,
		cast(clean_for_decimal_type(valor_base) AS DECIMAL(29,2)) AS valor_base,
		
		-- Fivetran metadata
		cast(_fivetran_index as bigint) as _fivetran_index,
		cast(_fivetran_deleted as boolean) as _fivetran_deleted,
		cast(_fivetran_synced as timestamp) as _fivetran_synced

FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.propostas_realizadas;

In [0]:
import pandas as pd

def parse_date(date):
    date_str = str(date).strip()
    
    formats_to_try = ['%d%m%Y',     # DDMMYYYY
                      '%d/%m/%Y',   # DD/MM/YYYY
                      '%d-%m-%Y',   # DD-MM-YYYY
                      '%d.%m.%Y',   # DD.MM.YYYY
                      '%Y-%m-%d',   # YYYY-MM-DD
                      '%m/%d/%Y']   # MM/DD/YYYY
    for fmt in formats_to_try:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue    
    # If none work, try the general approach
    try:
        return pd.to_datetime(date,
                              unit=date,
                              infer_datetime_format=True, # inspect the 1st few date strings and try to guess the date format
                              dayfirst=True, # It prefers to parse with day first (not strict)
                              errors='coerce') # returns NaT if it fails
    except Exception as e:
        return None

#Spark query runs/is distributed across the clusters, instead of Pandas (which runs on a single machine)
df = spark.table("workspace.sc_silver.propostas_realizadas") \
          .select("data_de_validade_da_proposta") \
          .where(r"NOT data_de_validade_da_proposta RLIKE '^\\d{4}-\\d{2}-\\d{2}$' "
                 r"AND NOT data_de_validade_da_proposta RLIKE '^\\d{2}-\\d{2}-\\d{4}$' "
                 r"AND data_de_validade_da_proposta IS NOT NULL")

pdf = df.toPandas()
pdf["data_de_validade_da_proposta"] = pdf["data_de_validade_da_proposta"].apply(parse_date)
#display(pdf[pdf["data_de_validade_da_proposta"].isna()])
display(pdf["data_de_validade_da_proposta"])

#pdf_excep = pdf[pdf["data_de_validade_da_proposta"].astype(str).str.startswith("EXCEP")]
#display(pdf_excep)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.viaturas 
SELECT * FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.viaturas;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_silver.viaturas_demo 
SELECT * 
  FROM workspace.sc_bronze_pbs2025bacapstoneg2u2.viaturas_demo;